<h1> AI CA2 REPORT </h1>
<h2>Atieh Armin - 810197648 </h2>
<p> In this project we are going to use Genetic Algorithm to implement a simple combinational logic circuit.
<br> To do this we have a truth table for the input that will be our constraints.We know that for n inputs in the truth table we will need n-1 logical gates and the inputs will connect to the gates in order.</p>
<h3> Implementation of the Chromosomes and Genes </h3>
<p> We defined every gate as a gene and the chromosome will be a sequence of genes.So we have 6 diffrenet kind of genes and the length of the chromosomes will be n-1.<br> We consider the AND gate as 1, OR gate as 2, NAND gate as 3, NOR gate as 4, XOR gate as 5 and XNOR gate as 6. For example a chromosome like [1,2,3] will represent a combinational circuit with 3 gates which the first and the second input will connect to an AND gate, the third input and the output of the AND gate will connect to an OR gate and the forth input and the output of the OR gate will connect to a NAND gate and the out put of this circuit will be the output of the NAND gate.</p>
    

In [1]:
import pandas as pd
import numpy as np
import random
import time


def initial_pop(pop_num, inputs):
    return np.random.randint(low=1, high=6, size=(pop_num, inputs))


def check_table(table,chromosome):
    result = []
    for row in table:
        i = 1
        out = row[0]
        for gene in chromosome:
            if gene == 1:  # and
                out = row[i] & out
            if gene == 2:  # or
                out = row[i] | out
            if gene == 3:  # nand
                out = not (row[i] & out)
            if gene == 4:  # nor
                out = not (row[i] | out)
            if gene == 5:  # xor
                out = row[i] ^ out
            if gene == 6:  # xnor
                out = not (row[i] ^ out)
            i += 1
        result.append(out)
    return result


def calculate_fitness(result, output):
    x = np.array([result, output], dtype=int)
    diff = np.diff(x, axis=0)
    return np.count_nonzero(diff == 0)

def sort_pop(fitness, population):
    inds = np.argsort(fitness)
    population = population[inds]
    fitnesses = np.take_along_axis(np.array(pop_fitness), inds, axis=None)
    return fitnesses, population

def selection(population, n):
    return population[len(population) - n:]


def crossover(parents):
    crossover_point = np.random.randint(1, len(parents[0]))
    children = []
    parents_cpy = []
    for k in range(len(parents)):
        chrom = []
        for m in range(len(parents[k])):
            chrom.append(parents[k][m])
        parents_cpy.append(chrom)

    while parents_cpy:
        child1 = np.hstack((parents[0][0:crossover_point], parents[1][crossover_point:])).tolist()
        child2 = np.hstack((parents[1][0:crossover_point], parents[0][crossover_point:])).tolist()
        children.append(child1)
        children.append(child2)
        parents_cpy.pop(0)
        parents_cpy.pop(0)
    return children


def mutation(parents,p):
    children = []
    parents_cpy = []
    for k in range(len(parents)):
        chrom = []
        for m in range(len(parents[k])):
            chrom.append(parents[k][m])
        parents_cpy.append(chrom)

    for i in range(len(parents_cpy)):
        chromosome = parents_cpy[i]
        for j in range(len(chromosome)):
            random.seed(time.time())
            new_gene = random.randint(1, 6)
            number = random.randint(1, 100)
            if number <= p:
                chromosome[j] = new_gene
        children.append(chromosome)
    return children


def new_generation(population):
    generation = []
    parents = selection(population, 10)
    generation.extend(parents)
    second_parents = selection(population[:len(population)-10],10)
    children = mutation(crossover(second_parents),60)
    generation.extend(children)
    mutated_children = mutation(parents,40)
    generation.extend(mutated_children)
    new_children = mutation(crossover(parents),20)
    generation.extend(new_children)
    return generation

def print_sol(chromosome):
    out = ""
    for gene in chromosome:
        if gene == 1:  # and
            out += "AND "
        if gene == 2:  # or
            out += "OR "
        if gene == 3:  # nand
            out += "NAND "
        if gene == 4:  # nor
            out += "NOR "
        if gene == 5:  # xor
            out += "XOR "
        if gene == 6:  # xnor
            out += "XNOR "
    print(out)

<h3>Functions</h3>
<h4> 1. Initial Population</h4>
<p> In this function we are going to make an initial population randomly. We are using numpy library to make some random values for our chromosomes.</p>

<h4> 2. Check Table</h4>
<p>The check table function is a function that will check the result of the chromosome for every row in the truth table.</p>
<h4> 3. Fitness</h4>
<p>We can define the fitness of a chromosome as the number of the cases it passed.In this case we will get an array of results that show the results of our chromosome for every line of our truth table and an array of the actual output- the output that is correct and is the last column of our truth table as inputs.Then we will calculate the discrete difference along the the 0 index of the arrays.After that we will count the zeros and we know that the number of the zeros represents the number of cases that our chromosome passed.</p>
<h4>4. Sort Population</h4>
<p>In this function we are sorting the population based on the fitness list.<br>np.argsort returns the indices that would sort the array and np.take_along_axis will take values from the input array by matching 1d index and data slices.</p>
<h4> 5. Selection</h4>
<p>The selection function is used in the first step of genetic algorithm after we calculate the fitness of every chromosome.It will select parents that we want to make the new generation.Here we will get the population wich is sorted by the fitness and the number of the parents we want as inputs.</p>
<h4> 6. Crossover </h4>
<p>Crossover is a function that will create a new set of chromosome by combining the parents.Here we used 1-point crossover and the crossover point is chosen randomly.</p>
<h4> 7. Mutation </h4>
<p>Mutation is a function that perfrom mutation which alters one or more gene values in a chromosome in the new population set generated. Mutation helps in getting more diverse oppourtinity.Here we mutate genes by the probability of p that can change in different situations.</p>
<h4> 8. New Generation </h4>
<p>In this function we make the new generation.We first select parents for the process of mutation and crossover.We copy the first 10 chromosomes exactly in the new generation.Then we mutate them with 40% probability and again add it to the new generation and then we crossover and mutate them with 20% probablity and then add it to the new generation.On the other hand we will crossover and mutate the next 10 chromosomes with 70% probability and add the children in the new generation.In this way we are improving our chromosomes.<br>The reason behind choosing these probability is that the first 10 chromosomes have better fitnesses and we want them to change a little and make better chromosomes but the next 10 chromosomes are not that good so we want to change them better and improve them.If we just mutate and crossover it's possible that we stock in a local maximum. To avoid this problem we chose different probability for the mutation and we chose the crossover point to be random.</p>
<h4>9. Print Solution</h4>
<p>In this function we decode the solution chromosome in to the gates and print the solution.</p>

In [2]:
t_table = pd.read_csv("truth_table.csv")

inputs = len(list(t_table))
output = list(t_table["Output"])
table = [t_table.columns.values.tolist()] + t_table.values.tolist()
table = table[1:]

<h3> Getting Input </h3>
<p>We have a truth table so we can read it with pandas library.Pandas.read_csv will return a dataframe.We cast this dataframe to list to optimize the timing of the program.</p>

In [3]:
pop_num = 40
population = initial_pop(pop_num, inputs - 2)

while True:
    pop_fitness = []
    for chromosome in population:
        fitness = calculate_fitness(check_table(table, chromosome), output)
        pop_fitness.append(fitness)
    fitnesses, population = sort_pop(pop_fitness,population)
    population = population.tolist()
    if fitnesses[len(fitnesses) - 1] == len(output):
        break
    population = np.array(new_generation(population))

print_sol(population[len(population)-1])

AND XOR NOR XOR AND OR AND XOR NOR 


<h3>Main</h3>
<p>First we define our initial population.We decided that the value of the population number be 40.We will run the algorithm with a while.For every generation we have a population of chromosomes.We will calculate the fitnesses for our population and then sort them.Then we check the maximum fitness if it is equal to 2^(gates+1) that is the length of the output list, it means we have reached to the solution so we break the while loop.In the end we print the solution.</p>
<h3> Output </h3>
<p>There is several solutions for every truth table.<br>We ran this program 2 times.<br>
    first output: "AND XOR OR XNOR AND OR NAND XNOR NOR " <br> 
    second output: "AND XOR NOR XOR AND OR AND XOR NOR " </p>
  